In [1]:
import pandas as pd 
import h5py as h5
import numpy as np

file = '/mnt/scratch/scratch/federico/MERGED_FEB2023/0-20300-0-37260_CEUAS_merged_v1.nc'

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)



/tmp/ipykernel_1613245/168431023.py:10: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


# TEST 
why in some files, given one record, we obtain different lat and lon combination (should not happen since we do not mix observations from different records)


In [2]:
dt =2538129600 # date to be tested 
pd.to_datetime( [dt] , unit='s',  origin=pd.Timestamp('1900-01-01') ) 

DatetimeIndex(['1980-06-06 12:00:00'], dtype='datetime64[ns]', freq=None)

In [3]:
# Example file 
f = h5.File(file, 'r')

In [4]:
def read_data(file, va=126, p=1000,  z_type=1, date_min=False, date_max='19550101'):
    #print('*** Reading ' , file)
    f = h5.File(file, 'r')
    ot = f['observations_table']

    dt = pd.to_datetime( f['recordtimestamp'][:] , unit='s',  origin=pd.Timestamp('1900-01-01') ).date 
                                                                     
    dic = {}
    for v in ['date_time' , 'z_coordinate' , 'z_coordinate_type' , 'observation_value' , 'observed_variable', 'latitude' , 'longitude', 'source_id']:
        if v not in ['source_id']:
        
            dic[v] = ot[v][:]
        else:
            values = [b''.join(i) for i in  ot[v][:] ] 
            dic[v] = values

    df = pd.DataFrame.from_dict(dic)
    
    df['date_time'] = pd.to_datetime( df['date_time'][:] , unit='s',  origin=pd.Timestamp('1900-01-01') ) 
    
    df = df.dropna( subset=['observation_value'])
    
    if va:
        df =df.loc[df.observed_variable == va]
    
    if p:
        #print('Filtering pressure')
        df =df.loc[(df.z_coordinate == p) & (df.z_coordinate_type == 1) ]

    if  z_type:
        df =df.loc[(df.z_coordinate_type ==  z_type) ]

    if date_max:
        df =df.loc[(df.date_time <= pd.Timestamp(year= int(date_max[0:4]), month=int(date_max[4:6]), day=int(date_max[6:8]), hour=int(date_max.split(':')[1]) ) ) ]
    if date_min :
        df =df.loc[(df.date_time >= pd.Timestamp(year= int(date_max[0:4]), month=int(date_max[4:6]), day=int(date_max[6:8]), hour=int(date_max.split(':')[1]) ) ) ]
        

    df.file = file.split('/')[-1]
    return df 

In [5]:
def read_header(file, date_min=False, date_max='19550101:12'):
    #print('*** Reading ' , file)
    f = h5.File(file, 'r')
    ot = f['header_table']

    dt = pd.to_datetime( ot['record_timestamp'][:] , unit='s',  origin=pd.Timestamp('1900-01-01') ) 
                                                                     
    dic = {}
    dic['report_timestamp'] = dt
    for v in ['latitude' , 'longitude', 'source_id' , 'duplicates']:
        if v not in ['source_id','duplicates']:
            dic[v] = ot[v][:]
        else:
            values = [b''.join(i) for i in  ot[v][:] ] 
            dic[v] = values

    for v in ['source_file']:
        values = [b''.join(i) for i in   f['source_configuration']['source_file'][:] ] 
        dic[v] = values
            
    df = pd.DataFrame.from_dict(dic)
    

    if date_max:
        df =df.loc[(df.report_timestamp <= pd.Timestamp(date_max)) ]
    if date_min :
        df =df.loc[(df.report_timestamp >= pd.Timestamp(date_min)) ]
        
    df.file = file.split('/')[-1]
    return df 


# I find that for the record with timestamp 2538129600 I get lats 42.87,
# 43.02 . Do you have an explanation for that?

In [6]:
h = read_header(file, date_min = '19800605' , date_max='19800607')

In [7]:
h

,report_timestamp,latitude,longitude,source_id,duplicates,source_file
50040,1980-06-05 00:00:00,42.869999,41.130001,b'era5_1',"b'10000001802,40046457762,40046457762,30000025902'",b'0-20300-0-37260_era5_1_harvested_era5.conv.??????.37260.txt.gz.nc'
50041,1980-06-05 06:00:00,43.020000,41.029999,b'era5_1',"b'10000001803,40046458619,40046458619,30000025903'",b'0-20300-0-37260_era5_1_harvested_era5.conv.??????.37260.txt.gz.nc'
50042,1980-06-05 12:00:00,43.020000,41.029999,b'era5_1',"b'10000001804,40046459062,40046459062,30000025904'",b'0-20300-0-37260_era5_1_harvested_era5.conv.??????.37260.txt.gz.nc'
50043,1980-06-05 18:00:00,42.869999,41.130001,b'era5_1',"b'10000001805,30000025905'",b'0-20300-0-37260_era5_1_harvested_era5.conv.??????.37260.txt.gz.nc'
50044,1980-06-06 00:00:00,43.020000,41.029999,b'era5_1',"b'10000001806,40046460397,40046460397,30000025906'",b'0-20300-0-37260_era5_1_harvested_era5.conv.??????.37260.txt.gz.nc'
50045,1980-06-06 06:00:00,43.020000,41.029999,b'era5_1',"b'10000001807,40046461260,40046461260,30000025907'",b'0-20300-0-37260_era5_1_harvested_era5.conv.??????.37260.txt.gz.nc'
50046,1980-06-06 12:00:00,43.020000,41.029999,b'era5_1',"b'10000001808,40046461708,40046461708,30000025908'",b'0-20300-0-37260_era5_1_harvested_era5.conv.??????.37260.txt.gz.nc'
50047,1980-06-06 18:00:00,42.869999,41.130001,b'era5_1',"b'10000001809,40046462665,40046462665,30000025909'",b'0-20300-0-37260_era5_1_harvested_era5.conv.??????.37260.txt.gz.nc'
50048,1980-06-07 00:00:00,42.869999,41.130001,b'era5_1',"b'10000001810,40046463117,40046463117,30000025910'",b'0-20300-0-37260_era5_1_harvested_era5.conv.??????.37260.txt.gz.nc'


In [8]:
df = read_data(file, va=126, p=False,  z_type=1, date_min = '19800606:12' , date_max='19800606:12')
### SEE that one observation has different lat/lon combination 

In [9]:
df

,date_time,z_coordinate,z_coordinate_type,observation_value,observed_variable,latitude,longitude,source_id
3141272,1980-06-06 12:00:00,1800.0,1,229.100006,126,43.020000,41.029999,b'era5_1'
3141274,1980-06-06 12:00:00,2000.0,1,226.100006,126,42.869999,41.130001,b'era5_1'
3141280,1980-06-06 12:00:00,3000.0,1,224.300003,126,42.869999,41.130001,b'era5_1'
3141290,1980-06-06 12:00:00,5000.0,1,219.899994,126,42.869999,41.130001,b'era5_1'
3141303,1980-06-06 12:00:00,7000.0,1,218.500000,126,42.869999,41.130001,b'era5_1'
3141311,1980-06-06 12:00:00,7520.0,1,213.300003,126,42.869999,41.130001,b'era5_1'
3141315,1980-06-06 12:00:00,8280.0,1,218.300003,126,42.869999,41.130001,b'era5_1'
3141319,1980-06-06 12:00:00,8680.0,1,215.100006,126,42.869999,41.130001,b'era5_1'
3141323,1980-06-06 12:00:00,9390.0,1,214.899994,126,42.869999,41.130001,b'era5_1'
3141332,1980-06-06 12:00:00,10000.0,1,217.100006,126,42.869999,41.130001,b'era5_1'


# CHECK ODB FILE 

In [10]:
"""
odb sql 'select DISTINCT date, time, lat, lon,varno WHERE date=19800606 AND time=120000'  -i era5.conv._37260 --no_alignment 
date@hdr	time@hdr	lat@hdr	lon@hdr	varno@body
19800606	120000	42.869999	41.130001	1
19800606	120000	42.869999	41.130001	2
19800606	120000	42.869999	41.130001	59
19800606	120000	42.869999	41.130001	29
19800606	120000	42.869999	41.130001	7
19800606	120000	42.869999	41.130001	111
19800606	120000	42.869999	41.130001	112
19800606	120000	42.869999	41.130001	3
19800606	120000	42.869999	41.130001	4
19800606	120000	42.869999	41.130001	39
19800606	120000	42.869999	41.130001	40
19800606	120000	42.869999	41.130001	58
19800606	120000	42.869999	41.130001	41
19800606	120000	42.869999	41.130001	42
19800606	120000	43.020000	41.029999	1
19800606	120000	43.020000	41.029999	2
19800606	120000	43.020000	41.029999	59
19800606	120000	43.020000	41.029999	29
19800606	120000	43.020000	41.029999	7
19800606	120000	43.020000	41.029999	111
19800606	120000	43.020000	41.029999	112
19800606	120000	43.020000	41.029999	3
19800606	120000	43.020000	41.029999	4
19800606	120000	43.020000	41.029999	39
19800606	120000	43.020000	41.029999	40
19800606	120000	43.020000	41.029999	58
19800606	120000	43.020000	41.029999	41
19800606	120000	43.020000	41.029999	42
"""

"\nodb sql 'select DISTINCT date, time, lat, lon,varno WHERE date=19800606 AND time=120000'  -i era5.conv._37260 --no_alignment \ndate@hdr\ttime@hdr\tlat@hdr\tlon@hdr\tvarno@body\n19800606\t120000\t42.869999\t41.130001\t1\n19800606\t120000\t42.869999\t41.130001\t2\n19800606\t120000\t42.869999\t41.130001\t59\n19800606\t120000\t42.869999\t41.130001\t29\n19800606\t120000\t42.869999\t41.130001\t7\n19800606\t120000\t42.869999\t41.130001\t111\n19800606\t120000\t42.869999\t41.130001\t112\n19800606\t120000\t42.869999\t41.130001\t3\n19800606\t120000\t42.869999\t41.130001\t4\n19800606\t120000\t42.869999\t41.130001\t39\n19800606\t120000\t42.869999\t41.130001\t40\n19800606\t120000\t42.869999\t41.130001\t58\n19800606\t120000\t42.869999\t41.130001\t41\n19800606\t120000\t42.869999\t41.130001\t42\n19800606\t120000\t43.020000\t41.029999\t1\n19800606\t120000\t43.020000\t41.029999\t2\n19800606\t120000\t43.020000\t41.029999\t59\n19800606\t120000\t43.020000\t41.029999\t29\n19800606\t120000\t43.020000\t41.0